In [30]:
import psutil
psutil.sensors_battery()
import pyttsx3 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

engine = pyttsx3.init() 
voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0"
# Use male voice 
engine.setProperty('voice', voice_id) 
newVoiceRate = 180
engine.setProperty('rate',newVoiceRate)

In [14]:
#data file namses
companies = ['HDFCBANK',
             'RELIANCE',
             'HDFC',
             'ICICIBANK',
             'INFY',
             'TCS',
             'KOTAKBANK',
             'ITC',
             'AXISBANK',
            'SBIN']
companies = np.array(companies)

In [ ]:
# saving company all year data with filter time between 9:16 to 15:30
for company in companies:
    data = pd.read_csv('/users/ajith/research/data/clean_data/'+ company+'.txt')
    data.drop(labels ='nan',axis = 1,inplace = True)
    data = data.iloc[:-2,:]
    data2 = data.copy()
    print('in '+company+'......')

#mask for every data files for removing mask time set
    masks = np.array(['15:4','15:31','15:32','15:33','15:34','15:5','15:51','15:55'
                      ,'15:56','15:57','15:58','16:36','09:08','16:','09:0','15:6','15:7'
                      ,'09:11','09:12','09:13','09:14','09:15','17:','18:','19:',
                      '20:','21:','22:','23:','24:'])
    for mask in masks:
        m = data2['Time'].str.find(mask).replace(-1,1).values.astype(bool) #removing 15:4 type time indices
        #m = data2['Time'].str.find(mask)
        data2 = data2[m]
    #checking at volume ==0
    #print(data2[data2['Volume']==0])
    #data2 = data2[data2['Volume']!=0]
    
    data2.to_csv(company+'.csv')
    print('data2 :',data2.shape)
    print('data : ',data.shape)
    print('difference: ',data.shape[0]-data2.shape[0])
    

In [15]:
# time frame creation between 9:16 to 15:30
#data checking
import pandas as pd

l = (pd.DataFrame(columns=['NULL'],
                  index=pd.date_range('2020-09-12T9:16', '2020-09-12T15:30',
                                      freq='1T'))
       .between_time('07:00','21:00')
       .index.strftime('%Y-%m-%dT%H:%M:%SZ')
       .tolist()
)
choice = 0
for i in range(len(l)):
    l[i] = l[i].replace('2020-09-12T','')
    l[i] = l[i].replace(':00Z','')
timeframe = np.array(l)
#Output:

In [70]:
# for for loop contents
# for bounding (creating) 15M data

years = np.array(['2017','2018','2019','2020'])
months = np.array(['0'+str(i+1) if i<9 else str(i+1) for i in range(12)])
dates = np.array(['0'+str(i+1) if i<9 else str(i+1) for i in range(31)])
times = []
for i in range(int(len(timeframe)/15)):
    times.append(timeframe[15*i:15*(i+1)])
times = np.array(times)
companies = np.array(['HDFCBANK',
             'RELIANCE',
             'HDFC',
             'ICICIBANK',
             'INFY',
             'TCS',
             'KOTAKBANK',
             'ITC',
             'AXISBANK',
            'SBIN'])
    

In [72]:
# creating(modifying 1min as 15m) 15min data by iterating through dates
# bounding 15 time frames like ['09:15','09:16',.....,'09:30']
# checked for data existance in timeframe in "temp" df
# append 15m timeframe data as one data in "data" df

missed = {}
engine.say('Hello Ajith Sir. Your programme started. Take a break. I will intimated you after the completion.')
#programme started
for company in companies:
    missed[company] = []
    old_data = pd.read_csv(company+'.csv')
    old_data['Date'] = old_data['Date'].astype(str)  #convering date format into string
    data = pd.DataFrame(columns = old_data.columns ) #main data file
    
    
    print('for %s company'%company)
    engine.say("programme for %s company started"%company) 
    engine.runAndWait() 
    
    for year in years:
        mask = old_data['Date'].str.find(year).replace(0,1).replace(-1,0).astype(bool)
        old_ = old_data[mask] #filtering year data and hold in old_
        print('in %s year'%year)
        engine.say('year {} in {} company'.format(year,company))
        engine.runAndWait()
        count = 0 #for audio out oof every 3 months
        for month in months:
            mask = old_['Date'].str.find(year+month).replace(0,1).replace(-1,0).astype(bool)
            old__ = old_[mask]    #filtering month data from old_
            print('in %s month'%month)
            count=count+1 # for audio out every 3 months(incremental)
            
            for day in dates:
                #print('in %s day'%day)
                old = old__[old__['Date']==year+month+day] #old certain date data
                #print(old)
                if len(old) == 0: # checking for date exsistance if no len is zero for df
                    print('{}/{}/{} not exist'.format(day,month,year))
                    continue;
                    
                for time in times:
                    temp = pd.DataFrame(columns = old.columns) #temporary data for store 15 time frames of date
                    '''for sub in time:
                        temp = temp.append(old[old['Time'] == sub],ignore_index=True)
                        if np.any(old['Time'] == sub)==False:
                            print('no data in {} .. {}'.format(year+month+day,sub))
                    if len(temp)==0:
                        print('no data in %s'%time[0])
                        continue;'''
                    temp = temp.append(old[old['Time'].isin(time)],ignore_index=True)
                    #print(temp)
                    if len(temp)==0:
                        print('no data in %s'%time[0])
                        # missed[company].append(day+month+year+' '+time)
                        continue;
                    data = data.append(pd.DataFrame({'Ticker': [company],
                                 'Date': [year+month+day],
                                 'Time': [time[0]],
                                 'Open': [temp['Open'][0]],
                                 'High': [np.max(temp['High'])],
                                 'Low': [np.min(temp['Low'])],
                                 'Close': [temp['Close'][len(temp)-1]],
                                 'Volume': [np.sum(temp['Volume'])]
                                })
                               )
                    #print(data)
                    
            print(data.shape)
            data.to_csv(company+'_15'+'.csv')
            if np.remainder(count,3)==0:
                engine.say('sucessfully completed 3 months Now in %s month'%month)
                engine.runAndWait() 
                
                if psutil.sensors_battery().power_plugged == False:
                    engine.say('Ajith sir. Batter weak. Please connect charger or Save the programme')
                    engine.runAndWait()
    print('end of %s'%company)
    engine.say('data for {} company completed. Please check data once'.format(company))
    engine.runAndWait()
    #data.to_csv(company+'new'+'.csv')
                        
                        
                        
                    
            

for HDFC company
in 2017 year
in 01 month
01/01/2017 not exist
07/01/2017 not exist
08/01/2017 not exist
14/01/2017 not exist
15/01/2017 not exist
21/01/2017 not exist
22/01/2017 not exist
26/01/2017 not exist
28/01/2017 not exist
29/01/2017 not exist
(525, 9)
in 02 month
04/02/2017 not exist
05/02/2017 not exist
11/02/2017 not exist
12/02/2017 not exist
18/02/2017 not exist
19/02/2017 not exist
24/02/2017 not exist
25/02/2017 not exist
26/02/2017 not exist
29/02/2017 not exist
30/02/2017 not exist
31/02/2017 not exist
(1000, 9)
in 03 month
04/03/2017 not exist
05/03/2017 not exist
11/03/2017 not exist
12/03/2017 not exist
13/03/2017 not exist
18/03/2017 not exist
19/03/2017 not exist
25/03/2017 not exist
26/03/2017 not exist
(1550, 9)
in 04 month
01/04/2017 not exist
02/04/2017 not exist
04/04/2017 not exist
08/04/2017 not exist
09/04/2017 not exist
14/04/2017 not exist
15/04/2017 not exist
16/04/2017 not exist
22/04/2017 not exist
23/04/2017 not exist
29/04/2017 not exist
30/04/2017 

In [9]:
for company in companies:
    print('in %s company'%company)
    for time in timeframe:
        data = pd.read_csv(company+'.csv')
        sample = np.sum(data['Time'] == time)
        print(' {} : {} '.format(time,sample))a
    
    #data.drop(labels ='nan',axis = 1,inplace = True)
    #data = data.iloc[:-2,:]
    #data2 = data.copy()
    
        
    
    

in HDFCBANK company
 09:31 : 901 
 09:32 : 901 
 09:33 : 901 
 09:34 : 901 
 09:35 : 901 
 09:36 : 901 
 09:37 : 901 
 09:38 : 901 
 09:39 : 901 
 09:40 : 901 
 09:41 : 901 
 09:42 : 901 
 09:43 : 901 
 09:44 : 901 
 09:45 : 901 
 09:46 : 901 
 09:47 : 901 
 09:48 : 901 
 09:49 : 901 
 09:50 : 901 
 09:51 : 901 
 09:52 : 901 
 09:53 : 901 
 09:54 : 901 
 09:55 : 901 
 09:56 : 901 
 09:57 : 901 
 09:58 : 901 
 09:59 : 901 
 10:00 : 901 
 10:01 : 900 
 10:02 : 900 
 10:03 : 900 
 10:04 : 900 
 10:05 : 900 
 10:06 : 900 
 10:07 : 900 
 10:08 : 900 
 10:09 : 900 
 10:10 : 900 
 10:11 : 899 
 10:12 : 899 
 10:13 : 899 
 10:14 : 900 
 10:15 : 899 
 10:16 : 900 
 10:17 : 900 
 10:18 : 900 
 10:19 : 900 
 10:20 : 900 
 10:21 : 900 
 10:22 : 901 
 10:23 : 901 
 10:24 : 901 
 10:25 : 901 
 10:26 : 900 
 10:27 : 900 
 10:28 : 901 
 10:29 : 901 
 10:30 : 901 
 10:31 : 901 
 10:32 : 901 
 10:33 : 901 
 10:34 : 901 
 10:35 : 901 
 10:36 : 901 
 10:37 : 901 
 10:38 : 901 
 10:39 : 901 
 10:40 : 901 


KeyboardInterrupt: 

In [4]:
masks = ['15:4','15:31','15:32','15:33','15:34','15:5','15:51','15:55','15:56','15:57','15:58','15:06',
         '16:36','09:08','16:','09:0']
for mask in masks:
    m = data2['Time'].str.find(mask).replace(-1,1).values.astype(bool) #removing 15:4 type time indices
    #m = data2['Time'].str.find(mask)
    data2 = data2[m]
    

NameError: name 'data2' is not defined